In [105]:
import os 
import requests

In [106]:
pdf_path = "world_geo.pdf"
if not os.path.exists(pdf_path):
    print('[INFO] The file does not exist, Downloading...')
    url  = "https://www.iipa.org.in/upload/world_geo.pdf"
    filename = pdf_path 
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, "wb") as file:
            file.write(response.content)
            print(f"[INFO] The file has been downloaded as  {filename}")
    else:
        print(f"[INFO] Failed to download the file. Status Code: {response.status_code}")
else:
    print("[INFO] File {pdf_path} already exists.")

[INFO] File {pdf_path} already exists.


PREPROCESS 

In [107]:
import fitz 
from tqdm.auto import tqdm 

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", "").strip()
    return cleaned_text 

def o_r_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_text = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_text.append({"page_number": page_number,
        "page_char_count" : len(text),
        "page_word_count" : len(text.split(" ")),
        "page_sentence_count" : len(text.split(". ")),
        "page_token_count" : len(text) / 4,
        "text": text}) # One token = 4 characters

    return pages_and_text

pages_and_text = o_r_pdf(pdf_path = pdf_path)
pages_and_text[:2]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 80,
  'page_word_count': 20,
  'page_sentence_count': 1,
  'page_token_count': 20.0,
  'text': '1      IIPA2022  22 World Geography  Short Answers PKP-03 by Dr Amit Kumar Singh'},
 {'page_number': 1,
  'page_char_count': 4596,
  'page_word_count': 163,
  'page_sentence_count': 43,
  'page_token_count': 1149.0,
  'text': '2  Contents 1.  Origin of Solar System and Sun ........................................................................................................... 6 2. Stars, Sun & Planets ............................................................................................................................. 8 Sun Statistics ....................................................................................................................................... 8 Solar Eclipse ........................................................................................................................................ 9 Lunar Eclipse .......

In [108]:
import random 

random.sample(pages_and_text, k=3)

[{'page_number': 37,
  'page_char_count': 2328,
  'page_word_count': 389,
  'page_sentence_count': 17,
  'page_token_count': 582.0,
  'text': "38  decline. When the height difference between the uplands and the valley bottoms reaches the maximum, slope decline becomes an important phenomenon. The slope of the upland begins to decline faster than the incision or dissection by any river stream. This effectively leads to the decrease in the relief features of height and the difference between the uplands and the bottom of valleys starts to lessen and diminish gradually. The Old Age Stage The stage following the mature stage is the old age' stage of relief development. In old age, the initially uplifted terrain feature gradually diminishes or reduces to a surface known as the peneplain. During the old age, which is the latest stage in the process, the erosion has been acting upon the terrain so long that although the terrain was at a significant height when uplifted, it is now reduced to a

In [109]:
import pandas as pd 

df = pd.DataFrame(pages_and_text)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,text
0,0,80,20,1,20.0,1 IIPA2022 22 World Geography Short Ans...
1,1,4596,163,43,1149.0,2 Contents 1. Origin of Solar System and Sun...
2,2,4448,208,40,1112.0,3 Types of Volcanoes ...........................
3,3,4560,203,44,1140.0,4 Continental Slope ............................
4,4,3474,178,31,868.5,5 Insolation or Incoming Solar Radiation .......


In [110]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count
count,129.00,129.00,129.00,129.00,129.00
mean,64.00,1655.45,262.89,16.95,413.86
std,37.38,732.36,96.85,7.75,183.09
min,0.00,80.00,20.00,1.00,20.00
25%,32.00,1146.00,189.00,12.00,286.50
50%,64.00,1699.00,264.00,16.00,424.75
75%,96.00,2055.00,341.00,21.00,513.75
max,128.00,4596.00,424.00,44.00,1149.00


In [111]:
from spacy.lang.en import English

nlp = English()
#Adding a sentencizer pipeline
nlp.add_pipe("sentencizer")

#Creating a documnet instance as an example

doc = nlp("I am a shitty person. I am a good man. I am ugly.")
assert len(list(doc.sents)) == 3

list(doc.sents)


[I am a shitty person., I am a good man., I am ugly.]

In [112]:
pages_and_text[0]

{'page_number': 0,
 'page_char_count': 80,
 'page_word_count': 20,
 'page_sentence_count': 1,
 'page_token_count': 20.0,
 'text': '1      IIPA2022  22 World Geography  Short Answers PKP-03 by Dr Amit Kumar Singh'}

In [113]:
for item in tqdm(pages_and_text):
    item["sentences"] = list(nlp(item["text"]).sents)

    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    item["page_sentence_count_spacy"] = len(item["sentences"])


  0%|          | 0/129 [00:00<?, ?it/s]

In [114]:
random.sample(pages_and_text, k=1)

[{'page_number': 126,
  'page_char_count': 2170,
  'page_word_count': 362,
  'page_sentence_count': 22,
  'page_token_count': 542.5,
  'text': "127  Greenhouse gases come from both human and natural sources. Gases like carbon dioxide, methane, and nitrous oxide naturally occur in the atmosphere. Others, such as chlorofluorocarbons (CFCs), are only produced by human activity. When short-wave radiation from the sun reaches Earth, most of it passes straight through and hits the surface. The Earth absorbs most of this radiation and gives off longer-wavelength infrared radiation. The greenhouse gases absorb some of this infrared radiation, instead of it passing straight out into space. The atmosphere then emits radiation in all directions, sending some of it back to the surface, causing the planet to heat up. This process is known as the 'greenhouse effect'. The greenhouse effect is critical to our survival. In fact, without greenhouse gases, Earth would be about 30 degrees colder than it i

In [115]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy
count,129.00,129.00,129.00,129.00,129.00,129.00
mean,64.00,1655.45,262.89,16.95,413.86,15.51
std,37.38,732.36,96.85,7.75,183.09,6.25
min,0.00,80.00,20.00,1.00,20.00,1.00
25%,32.00,1146.00,189.00,12.00,286.50,10.00
50%,64.00,1699.00,264.00,16.00,424.75,16.00
75%,96.00,2055.00,341.00,21.00,513.75,20.00
max,128.00,4596.00,424.00,44.00,1149.00,31.00


CHUNKING : Splitting sentecnces in groups of 10 or less

In [116]:
num_sentence_chunk_size = 10 
def split_list(input_list: list, slice_size: int = num_sentence_chunk_size) -> list[list[str]]:
    return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

text_list = list(range(70))
split_list(text_list)


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]]

In [117]:
for item in tqdm(pages_and_text):
    item["sentence_chunks"] = split_list(input_list=item["sentences"], slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/129 [00:00<?, ?it/s]

In [118]:
random.sample(pages_and_text, k = 1)

[{'page_number': 108,
  'page_char_count': 1699,
  'page_word_count': 287,
  'page_sentence_count': 18,
  'page_token_count': 424.75,
  'text': '109  28. Atmospheric Circulation and Weather System  Variying temperature on earth plays an important role in the uneven distribution over the surface of the earth. Air expands when heated and gets compressed when cooled. This results in variations in the atmospheric pressure. The result is that it causes the movement of air from high pressure to low pressure, setting the air in motion. You already know that air in horizontal motion is wind. Atmospheric pressure also determines when the air will rise or sink. The wind redistributes the heat and moisture across the planet, thereby, maintaining a constant temperature for the planet as a whole. The vertical rising of moist air cools it down to form the clouds and bring precipitation. Atmospheric Pressure Atmospheric pressure also determines when the air will rise or sink. The wind redistributes t

In [119]:
df = pd.DataFrame(pages_and_text)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,page_token_count,page_sentence_count_spacy,num_chunks
count,129.00,129.00,129.00,129.00,129.00,129.00,129.00
mean,64.00,1655.45,262.89,16.95,413.86,15.51,1.97
std,37.38,732.36,96.85,7.75,183.09,6.25,0.71
min,0.00,80.00,20.00,1.00,20.00,1.00,1.00
25%,32.00,1146.00,189.00,12.00,286.50,10.00,1.00
50%,64.00,1699.00,264.00,16.00,424.75,16.00,2.00
75%,96.00,2055.00,341.00,21.00,513.75,20.00,2.00
max,128.00,4596.00,424.00,44.00,1149.00,31.00,4.00


In [120]:
import re 

pages_and_chunks = []
for item in tqdm(pages_and_text):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # join the sentences together into paragraph like structure aka join the list of sentences into one paragraph 

        joined_sentence_chunk = "".join(sentence_chunk).replace("  "," ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)

        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4

        pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/129 [00:00<?, ?it/s]

254

In [121]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 12,
  'sentence_chunk': 'They form semi-circles. They are also drawn at an interval of 10. These meridians of longitude and parallels of latitude form a network which is called grid. Unlike the parallels of latitudes, the meridians of longitudes are equal in length. As one go from the equator to the poles the distance between two meridians decreases. There was an agreement world over that the longitude passing through Greenwich Observatory near the city of London will be considered as the Prime Meridian. It is considered to be 00 Longitude and from it we count 1800 eastward as well as 1800 westward. It is interesting that 1800 east and 1800 west meridians are the same line. To',
  'chunk_char_count': 660,
  'chunk_word_count': 114,
  'chunk_token_count': 165.0}]

In [122]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,254.00,254.00,254.00,254.00
mean,62.39,838.00,131.24,209.50
std,37.55,527.47,60.81,131.87
min,0.00,5.00,1.00,1.25
25%,29.25,600.50,95.25,150.12
50%,61.00,832.50,137.50,208.12
75%,94.00,1050.25,174.00,262.56
max,128.00,4446.00,321.00,1111.50


In [123]:
min_token_length = 15
for index, row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    # Example processing code
    print(f"Processing row {index}: {row.to_dict()}")

Processing row 188: {'page_number': 93, 'sentence_chunk': 'It is largely responsible', 'chunk_char_count': 25, 'chunk_word_count': 4, 'chunk_token_count': 6.25}
Processing row 173: {'page_number': 85, 'sentence_chunk': 'It is the layer of the ocean', 'chunk_char_count': 28, 'chunk_word_count': 7, 'chunk_token_count': 7.0}
Processing row 145: {'page_number': 70, 'sentence_chunk': 'wide.', 'chunk_char_count': 5, 'chunk_word_count': 1, 'chunk_token_count': 1.25}
Processing row 117: {'page_number': 56, 'sentence_chunk': 'The deepest section is less than 55m (180 feet) underwater.', 'chunk_char_count': 59, 'chunk_word_count': 10, 'chunk_token_count': 14.75}
Processing row 17: {'page_number': 8, 'sentence_chunk': 'The earth is the only planet where some special', 'chunk_char_count': 47, 'chunk_word_count': 9, 'chunk_token_count': 11.75}


In [124]:
pages_and_chunks_over_min_token_length = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_length[:2]

[{'page_number': 0,
  'sentence_chunk': '1   IIPA2022 22 World Geography Short Answers PKP-03 by Dr Amit Kumar Singh',
  'chunk_char_count': 75,
  'chunk_word_count': 15,
  'chunk_token_count': 18.75},
 {'page_number': 1,
  'sentence_chunk': '2 Contents 1. Origin of Solar System and Sun ........................................................................................................... 6 2. Stars, Sun & Planets ............................................................................................................................. 8 Sun Statistics ....................................................................................................................................... 8 Solar Eclipse ........................................................................................................................................ 9 Lunar Eclipse ..................................................................................................................................

In [125]:
random.sample(pages_and_chunks_over_min_token_length, k=1)

[{'page_number': 98,
  'sentence_chunk': '99 26. Insolation and Heat Balance of the Earth The atmosphere is essential for the survival of plant and animal life. They also require the optimum temperature to keep themselves warm and grow. In this article, we are going to discuss in details about some of the topics like insolation, terrestrial radiation, heat budget of the earth, albedo, etc. Insolation or Incoming Solar Radiation \uf0b7 As we all know, the sun is the primary source of energy for the earth. The sun radiates its energy in all directions into space in short wavelengths, which is known as solar radiation.\uf0b7 The earth’s surface receives only a part of this radiated energy (2 units out of 1,00,00,00,000 units of energy radiated by the sun).\uf0b7 The energy received by the earth’s surface in the form of short waves is termed as Incoming Solar Radiation or Insolation.\uf0b7 The amount of insolation received on the earth’s surface is far less than that is radiated from the s

Embedding : 

A useful numerical representation and it is a learned representation. 


{
    the: 0,
    a : 1,
    ...
}

In [126]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-MiniLM-L6-v2", device="cuda")

#create a list of sentences
sentences = ["The sentence transformer provides an easier way to create embeddings",
"Sentences can be embedded one by one in  a list.","I like chimps!"]

embeddings = embedding_model.encode(sentences)
embedding_dict = dict(zip(sentences, embeddings))

for sentence, embedding in embedding_dict.items():
    print(f"Sentence: {sentence}")
    print(f"Embedding: {embedding}")


C:\Users\aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sentence: The sentence transformer provides an easier way to create embeddings
Embedding: [-6.87679201e-02 -3.37028950e-02  3.68130989e-02  1.77186299e-02
  4.59097028e-02  7.25906417e-02 -2.06882395e-02  5.50199710e-02
  9.44427997e-02 -4.44253236e-02  5.07963374e-02  1.03461444e-02
  1.73806529e-02  6.51760772e-02  1.94573663e-02  8.53721723e-02
  1.59757342e-02  1.02003135e-01 -7.83813596e-02 -1.06738113e-01
  6.44138968e-03  3.28693874e-02  2.74742004e-02 -6.96152002e-02
  5.03769144e-02  6.34706244e-02 -3.52961123e-02  1.27846748e-02
  7.83551186e-02 -6.58562314e-03 -1.30270229e-04 -5.40848039e-02
 -4.96062152e-02  9.75598395e-02 -3.67665216e-02  2.29739249e-02
  4.46881577e-02  7.04804584e-02 -4.62974198e-02 -1.74799338e-02
  2.25367267e-02  6.00746088e-03  6.89678416e-02  5.03606573e-02
  4.12279367e-02 -3.38686816e-02 -4.16970365e-02 -9.08160862e-03
 -2.03304663e-02  1.73994489e-02 -4.90085185e-02 -2.83182859e-02
 -9.14116317e-05  6.86822906e-02 -4.36346792e-02  3.69178839e-02


In [127]:
embeddings[0].shape

(384,)

In [128]:
embedding = embedding_model.encode("My favourite animal is the donkey")
embedding

array([-5.49600944e-02,  7.83105660e-03,  4.53742221e-02,  1.39820976e-02,
        6.11190312e-03,  3.69757041e-02,  5.68162575e-02,  8.16690922e-03,
        6.46038875e-02,  3.74894068e-02, -3.10413763e-02, -6.54169694e-02,
        1.80261582e-02,  6.76760450e-02,  4.60805856e-02, -7.25531066e-03,
        2.34833471e-02, -2.68905889e-02,  1.29957236e-02, -7.20567480e-02,
       -9.44685787e-02,  5.52156903e-02,  1.91131514e-02, -3.16299610e-02,
       -1.05778761e-01, -7.45278820e-02,  8.34870897e-03,  2.31317915e-02,
       -3.54249147e-04, -7.11231530e-02, -5.96878007e-02, -5.54633066e-02,
        5.56221940e-02, -1.30626876e-02, -3.61372605e-02,  3.70618515e-02,
        2.22166744e-03, -6.46472052e-02,  8.49302113e-02,  5.51789813e-02,
        2.26267688e-02,  1.08027216e-02,  5.81755601e-02, -3.67338434e-02,
        4.32512118e-03,  3.97136435e-02, -4.81586643e-02, -4.30390388e-02,
        8.91654491e-02,  2.15335749e-02,  1.61640681e-02, -1.13695599e-02,
       -2.48992108e-02, -

In [129]:
%%time 

embedding_model.to("cuda")

for item in tqdm(pages_and_chunks_over_min_token_length):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])



  0%|          | 0/249 [00:00<?, ?it/s]

CPU times: total: 1.31 s
Wall time: 1.5 s


In [130]:
%%time 

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_length]
text_chunks[100]

CPU times: total: 0 ns
Wall time: 0 ns


'Deflation Hollows Deflation is the removal of loose particles from the ground by the action of wind. When deflation causes a shallow depression by persistent movements of wind, they are called as deflation hollows. Mushroom Tables Ventifacts are rocks that have been abraded, pitted, etched, grooved, or polished by wind-driven sand or ice crystals. These geomorphic features are most typically found in arid environments where there is little vegetation to interfere with aeolian particle transport, where there are frequently strong winds, and where there is a steady but not overwhelming supply of sand. Mushroom Tables / Mushroom rocks are Ventifacts in the shape of a mushroom. In deserts, a greater amount of sand and rock particles are transported close to the ground by the winds which cause more bottom erosion in overlying rocks than the top. This'

In [131]:
len(text_chunks)

249

In [132]:
%%time 

text_chunk_embeddings = embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)
text_chunk_embeddings

CPU times: total: 188 ms
Wall time: 288 ms


tensor([[ 0.0561,  0.0737,  0.0419,  ..., -0.0218, -0.1246,  0.0038],
        [-0.0754,  0.0494, -0.0252,  ...,  0.0150, -0.0143,  0.0286],
        [ 0.0264, -0.0255,  0.0595,  ..., -0.0240, -0.0227, -0.0092],
        ...,
        [-0.0200,  0.0131,  0.1068,  ..., -0.0089,  0.0557, -0.0017],
        [-0.0557,  0.0520,  0.1518,  ..., -0.0607, -0.0696, -0.0524],
        [-0.0396,  0.0280,  0.1182,  ..., -0.0675, -0.1298, -0.0064]],
       device='cuda:0')

In [133]:
## Saving embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_length)
embeddings_df_save_path = " text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [134]:
## viewing the saved file

text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embeddings_df_load.head()


,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,0,1 IIPA2022 22 World Geography Short Answers ...,75,15,18.75,[ 5.60791939e-02 7.36621842e-02 4.18552086e-...
1,1,2 Contents 1. Origin of Solar System and Sun ....,3711,128,927.75,[-7.53976032e-02 4.93803062e-02 -2.51827892e-...
2,1,"Temperature, Pressure and Density of the Earth...",879,30,219.75,[ 2.63881050e-02 -2.54815072e-02 5.94870374e-...
3,2,3 Types of Volcanoes ............................,4446,206,1111.50,[ 7.03721046e-02 -3.47200967e-02 5.47501445e-...
4,3,4 Continental Slope .............................,3865,167,966.25,[ 1.10680657e-02 4.12390829e-04 -1.85156371e-...


RAG : Search and Answer :: 

Retrieve relevant passages based on Query and use those passages to augment an input to an LLM so it can generate an output based on those relevant passages.

In [140]:
 #Simimlarity Search 
import torch 
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"
import pandas as pd

text_chunks_and_embeddings_df_load["embedding"] = text_chunks_and_embeddings_df_load["embedding"].apply(lambda x: np.array([float(i) for i in str(x).strip("[]").split()]))

embeddings = torch.tensor(np.stack(text_chunks_and_embeddings_df_load["embedding"].tolist(), axis=0), dtype=torch.float32).to(device)

pages_and_chunks =  text_chunks_and_embeddings_df_load.to_dict(orient="records")

text_chunks_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,0,1 IIPA2022 22 World Geography Short Answers ...,75,15,18.75,"[0.0560791939, 0.0736621842, 0.0418552086, 0.0..."
1,1,2 Contents 1. Origin of Solar System and Sun ....,3711,128,927.75,"[-0.0753976032, 0.0493803062, -0.0251827892, 0..."
2,1,"Temperature, Pressure and Density of the Earth...",879,30,219.75,"[0.026388105, -0.0254815072, 0.0594870374, 0.0..."
3,2,3 Types of Volcanoes ............................,4446,206,1111.50,"[0.0703721046, -0.0347200967, 0.0547501445, -0..."
4,3,4 Continental Slope .............................,3865,167,966.25,"[0.0110680657, 0.000412390829, -0.0185156371, ..."


In [141]:
embeddings.shape

#259 embeddings all of size 384

torch.Size([249, 384])

In [142]:
from sentence_transformers import util, SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-MiniLM-L6-v2", device=device)


C:\Users\aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SEMANTIC SEARCH PIPELINE

1. Defining a Query String
2. Turn the query into an embedding. 
3. Perform a dot product or cosine simmilarity function between the text embeddings and the query embedding. 
4. Sort the results from 3 in descending order.

In [147]:
# Defining the Query String 

query = "rivers and lakes"
print(f"query: {query}")

# turning the query into an embedding using the same model

query_embedding = embedding_model.encode(query, convert_to_tensor=True).to("cuda")

# getting simmilarity scores using the dot product 


dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
results_dot_product = torch.topk(dot_scores, k=5)
results_dot_product

query: rivers and lakes


torch.return_types.topk(
values=tensor([0.5441, 0.4873, 0.4794, 0.4749, 0.4547], device='cuda:0'),
indices=tensor([ 80, 106,  79,  78,  87], device='cuda:0'))

In [148]:
#checking for working 
pages_and_chunks[80]

{'page_number': 40,
 'sentence_chunk': '41 many tributaries. The work of river predominantly becomes transportation of the eroded materials from the upper course (little deposition too). Landforms like alluvial fans, piedmont alluvial plains, meanders etc.can be seen at this stage. Lower Course/ Stage of Old (Deposition dominates): The river starts to flow through a broad, level plain with heavy debris brought down from upper and middle courses. Vertical erosion has almost stopped and lateral erosion still goes on. The work of the river is mainly deposition, building up its bed and forming an extensive flood plain. Landforms like braided channels, floodplains, levees, meanders, oxbow lakes, deltas etc.can be seen at this stage. Running water: erosion, transportation, and deposition Erosion occurs when overland flow moves soil particles downslope.',
 'chunk_char_count': 818,
 'chunk_word_count': 122,
 'chunk_token_count': 204.5,
 'embedding': array([ 4.39054668e-02, -2.59029903e-02,  1.